In [1]:
# 1. Library imports
import pandas as pd
from IPython.display import Markdown, display
from extraction import ExtractionAgent
from ppi_deprescribe import merge_results, ppi_deprescribe
import os 
from pathlib import Path 


f:\LangChain\DataSci210_MedicationDeprescriber\src\extraction.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
f:\LangChain\.venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:

path = Path('F:/LangChain/data')
groq = os.environ['groqkey']
pd.set_option('display.max_colwidth', None)

In [3]:

def deprescribe(key):
    # extract information
    llm_agent = ExtractionAgent(groq_key=groq, data_path=path)

    results_dict = {
        "diagnosis_dict": llm_agent.extract_diagnosis(encounter_key=key),
        "encounter_dict": llm_agent.extract_encounter_info(encounter_key=key),
        # Is the reasoning in the json or sepearte?
        # Should the reasoning be included in any of them or just the diangosis with the reasoning seperate?
        "notes_dict": llm_agent.extract_notes(encounter_key=key),
    }
    print(results_dict['notes_dict'])
    # # #   master formatter step   # # #
    # merge the diagnosis booleans (just use OR logic for now)
    # make a final "reasoning" behind the recommendation
    final_dict = merge_results(results_dict=results_dict)

    # feed the three reasonings to LLM to get a single summary
    final_reasoning = llm_agent.summarize_reasonings(results_dict=results_dict)

    # # #   get recommendation from PPI algorithm   # # #
    recommendation_str = ppi_deprescribe(patient_diagnosis=final_dict)
    return recommendation_str, final_reasoning
    

In [ ]:
enc_key = 'D8017C77BA15FA'

In [4]:
labeled_data = pd.read_csv('F:/LangChain/data/LabeledResponses.csv')
labeled_data = labeled_data.head(1)
labeled_data.head()

,key,reason,recommendation,GS_response,Note
0,D6253A5CE371EA,example with gastrointestinal bleed should continue PPI,continue,"This patient has a history of a gastrointestinal bleeding which was present on the admission diagnosis. The pantoprazole 20mg taken twice daily should be continued. The GI bleed is mentioned in the note from 09/14/2023, on 09/17/2023, and on 09/20/2023.",NaN


In [5]:
def apply_deprescribe(row):
    try:
        recommendation_str, final_reasoning = deprescribe(row['key'])
        return pd.Series([recommendation_str, final_reasoning])
    except:
        return pd.Series([None, None])

labeled_data[['Rec_Returned', 'Reasoning']] = labeled_data.apply(apply_deprescribe, axis=1)

{'Mild to moderate esophagitis': False, 'GERD': False, 'Peptic Ulcer Disease': False, 'Upper GI symptoms': False, 'ICU Stress Ulcer Prophylaxis': False, 'Barretts Esophagus': False, 'Chronic NSAID use with bleeding risk': False, 'Severe esophagitis': False, 'Documented history of bleeding GI ulcer': True, 'H pylori infection': False, 'Reasoning': "The patient has a history of mild gastritis and small AVMs, but no active bleeding was found during the endoscopy. The patient's recent stroke and SAH make them a high-risk candidate for anesthesia and endoscopy. The patient's AC has been limited by slow ooze in the past, but it is recommended to restart necessary AC and AP under close observation."}


In [6]:
labeled_data.head()

,key,reason,recommendation,GS_response,Note,Rec_Returned,Reasoning
0,D6253A5CE371EA,example with gastrointestinal bleed should continue PPI,continue,"This patient has a history of a gastrointestinal bleeding which was present on the admission diagnosis. The pantoprazole 20mg taken twice daily should be continued. The GI bleed is mentioned in the note from 09/14/2023, on 09/17/2023, and on 09/20/2023.",NaN,continue,"The patient has a documented history of gastrointestinal bleeding (GIB) or gastrointestinal hemorrhage, which suggests a history of bleeding GI ulcer. There is no specific information provided about other conditions such as esophagitis, GERD, Peptic Ulcer Disease, Barrett's Esophagus, or H pylori infection, so they are marked as false. The patient's medical history, including recent stroke and SAH, and presence of small AVMs, may suggest a higher risk for certain conditions, but without further information, these conditions cannot be confirmed."


In [7]:
labeled_data.to_csv('F:/LangChain/data/LabeledResponses_ReturnedResponse.csv', index=False)